# National Insurance number allocations to adult overseas nationals to March 2018

Convert spreadsheet from

https://www.gov.uk/government/statistics/national-insurance-number-allocations-to-adult-overseas-nationals-to-march-2018

Table 1: NINo registrations to adult overseas nationals entering the UK by quarter of registration and world region

In [1]:
from gssutils import *
scraper = Scraper('https://www.gov.uk/government/statistics/' \
                  'national-insurance-number-allocations-to-adult-overseas-nationals-to-march-2018')
scraper

## National Insurance number allocations to adult overseas nationals to March 2018

National Insurance number allocations to adult overseas nationals entering the UK to March 2018.

### Description

We also publish data on the [nationality of DWP working age benefit claimants
at the point of National Insurance number
registration](https://www.gov.uk/government/statistics/nationality-at-point-
of-nino-registration-of-dwp-working-age-benefit-recipients-data-to-feb-2017).

This quarterly report contains data on National Insurance number allocations
to adult overseas nationals entering the UK.

The summary tables, derived from Stat-Xplore, show National Insurance number
allocations to adult overseas nationals entering the UK by:

  * quarter of registration and world region – January 2002 to March 2018
  * region and local authority by world area – registrations year to March 2018
  * registrations by nationality – year to March 2018

### Explore the statistics with our interactive tools

Full statistics on National Insurance number allocations to adult overseas
nationals entering the UK are available from [Stat-Xplore](https://stat-
xplore.dwp.gov.uk/).

View geographical breakdowns of National Insurance number registrations using
our interactive maps:

  * [world and Great Britain (GB) maps](https://ninodash.herokuapp.com/) showing National Insurance number registrations by nationality for each GB region
  * [local authority map](http://dwp-stats.maps.arcgis.com/apps/Viewer/index.html?appid=e449d3f2bd5e4e55aebc6154f69de07a) showing National Insurance number registrations from different world areas by GB local authority

Next release of these statistics: August 2018.

  *[GB]: Great Britain



### Distributions

1. National Insurance number allocations to adult overseas nationals to March 2018 ([application/pdf](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/710324/nino-registrations-adult-overseas-nationals-march-2018.pdf))
1. Summary: National Insurance number allocations to adult overseas nationals to March 2018 ([application/pdf](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/710070/nino-registrations-adult-overseas-nationals-march-2018-summary.pdf))
1. Summary tables: National Insurance number allocations to adult overseas nationals to March 2018 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/710047/nino-registrations-adult-overseas-nationals-march-2018-tables.ods))


In [2]:
sheets = scraper.distribution(
    title='Summary tables: National Insurance number allocations to adult overseas nationals to March 2018',
    mediaType=ODS
).as_databaker()
[t.name for t in sheets]

['CONTENTS', '1', '2', '3', '4']

The tab/sheet named '1' has "Table 1: NINo registrations to adult overseas nationals entering the UK by quarter of registration and world region"

In [3]:
savepreviewhtml(sheets[1])

,,,,,,,,,,,,,,,,
Table 1: NINo registrations to adult overseas nationals entering the UK by quarter of registration and world region,,,,,,,,,,,,,,,,
Note : Caution should be taken in interpreting trends based on quarterly data,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,
,,European Union,,,,Non-European Union (Other Europe),Asia,,,,Rest of the World,,,,,Unknown
,TOTAL,European Union EU15,European Union EU8,European Union EU2,European Union Other,Other Europe,Middle East and Central Asia,East Asia,South Asia,South East Asia,Sub-Saharan Africa,North Africa,North America,Central and South America,Oceania,Unknown
,,,,,,,,,,,,,,,,
Jan-02 to Mar-02,69701.0,18150.0,2217.0,1269.0,455.0,3114.0,4411.0,2390.0,10591.0,4063.0,11376.0,757.0,2770.0,2152.0,5902.0,91.0
Apr-02 to Jun-02,56619.0,13416.0,1725.0,1124.0,287.0,2452.0,4600.0,2030.0,9443.0,3407.0,9198.0,661.0,1903.0,1919.0,4363.0,97.0
Jul-02 to Sep-02,94436.0,20844.0,2912.0,1563.0,410.0,3732.0,6900.0,3381.0,15496.0,5231.0,17555.0,1024.0,3079.0,3922.0,8207.0,176.0
Oct-02 to Dec-02,90535.0,21525.0,2664.0,1323.0,341.0,3512.0,5845.0,3430.0,14618.0,5253.0,16465.0,940.0,3344.0,3658.0,7288.0,330.0


In [4]:
observations = sheets[1].excel_ref('B8').expand(DOWN).expand(RIGHT).is_not_blank()
area = sheets[1].excel_ref('B6').expand(RIGHT).is_not_blank()
period = sheets[1].excel_ref('A8').expand(DOWN).is_not_blank()
period = period - period.regex('^INFO').expand(DOWN)
dimensions = [
    HDim(period,'Period',DIRECTLY,LEFT),
    HDim(area,'Nationality', DIRECTLY, ABOVE),
    HDimConst('Measure Type', 'Count'),
    HDimConst('Unit','People')
]
cs = ConversionSegment(observations, dimensions, processTIMEUNIT=True)
savepreviewhtml(cs)

In [5]:
table1 = cs.topandas()
table1.head()

,OBS,Period,Nationality,Measure Type,Unit
0,69701.0,Jan-02 to Mar-02,TOTAL,Count,People
1,18150.0,Jan-02 to Mar-02,European Union EU15,Count,People
2,2217.0,Jan-02 to Mar-02,European Union EU8,Count,People
3,1269.0,Jan-02 to Mar-02,European Union EU2,Count,People
4,455.0,Jan-02 to Mar-02,European Union Other,Count,People


In [6]:
table1['Value'] = table1['OBS'].astype(int)
table1 = table1[['Period','Nationality','Measure Type','Value','Unit']].copy()
table1.head()

,Period,Nationality,Measure Type,Value,Unit
0,Jan-02 to Mar-02,TOTAL,Count,69701,People
1,Jan-02 to Mar-02,European Union EU15,Count,18150,People
2,Jan-02 to Mar-02,European Union EU8,Count,2217,People
3,Jan-02 to Mar-02,European Union EU2,Count,1269,People
4,Jan-02 to Mar-02,European Union Other,Count,455,People


Periods are "British Calendar" quarters, so convert e.g. `Jan-02 to Mar-02` to `quarter/2002-Q1`

In [7]:
import re
quarter_re = re.compile('\*?(Jan|Apr|Jul|Oct)-([0-9]{2}) to (Mar|Jun|Sep|Dec)-([0-9]{2})')
def to_quarter(s):
    match = quarter_re.match(s)
    assert match, f"period {s} doesn't match pattern"
    assert match.group(2) == match.group(4), "quarter start and end must be in the same year"
    if match.group(1) == 'Jan':
        assert match.group(3) == 'Mar', 'Quarter starting in Jan should end in Mar'
        return f"quarter/20{match.group(2)}-Q1"
    elif match.group(1) == 'Apr':
        assert match.group(3) == 'Jun', 'Quarter starting in Apr should end in Jun'
        return f"quarter/20{match.group(2)}-Q2"
    elif match.group(1) == 'Jul':
        assert match.group(3) == 'Sep', 'Quarter starting in Jul should end in Sep'
        return f"quarter/20{match.group(2)}-Q3"
    elif match.group(1) == 'Oct':
        assert match.group(3) == 'Dec', 'Quarter starting in Oct should end in Dec'
        return f"quarter/20{match.group(2)}-Q4"

table1['Period'] = table1['Period'].map(to_quarter)

Table 2: Adult overseas nationals entering the UK by Region / Local Authority and world region

In [8]:
savepreviewhtml(sheets[2])

,,,,,,,,,,,,,,,,,
Table 2: NINo registrations to adult overseas nationals entering the UK by Region / Local Authority and world region - registrations year to March 2018,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,
,,,European Union,,,,Non-European Union (Other Europe),Asia,,,,Rest of the World,,,,,Unknown
,,TOTAL,European Union EU15,European Union EU8,European Union EU2,European Union Other,Other Europe,Middle East and Central Asia,East Asia,South Asia,South East Asia,Sub-Saharan Africa,North Africa,North America,Central and South America,Oceania,Unknown
,,,,,,,,,,,,,,,,,
,Region,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,
E12000001,North East,9825.0,1985.0,1200.0,2152.0,95.0,220.0,823.0,422.0,1024.0,593.0,578.0,211.0,233.0,139.0,128.0,17.0
E12000002,North West,49553.0,12049.0,9923.0,11398.0,206.0,808.0,2386.0,1345.0,5274.0,1008.0,2469.0,898.0,576.0,585.0,553.0,72.0
E12000003,Yorkshire and The Humber,35693.0,5473.0,8373.0,10714.0,155.0,530.0,1537.0,834.0,3626.0,788.0,1705.0,617.0,466.0,396.0,420.0,57.0


In [9]:
observations = sheets[2].excel_ref('C9').expand(DOWN).expand(RIGHT).is_not_blank() - sheets[2].excel_ref('A432').expand(RIGHT)
area = sheets[2].excel_ref('C5').expand(RIGHT).is_not_blank()
RLa = sheets[2].excel_ref('A9').expand(DOWN).is_not_blank()

Dimensions = [
            HDim(RLa,'Geography',DIRECTLY,LEFT),
            HDim(area,'Nationality', DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]
cs = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
savepreviewhtml(cs)

In [10]:
table2 = cs.topandas()
table2.head()

,OBS,DATAMARKER,Geography,Nationality,Measure Type,Unit
0,9825,NaN,E12000001,TOTAL,Count,People
1,1985,NaN,E12000001,European Union EU15,Count,People
2,1200,NaN,E12000001,European Union EU8,Count,People
3,2152,NaN,E12000001,European Union EU2,Count,People
4,95,NaN,E12000001,European Union Other,Count,People


Drop non-numeric observations and ignore data markers.

Todo: figure out how to represent missing measured values.

In [11]:
import numpy as np
table2['OBS'].replace('', np.nan, inplace=True)
table2.dropna(subset=['OBS'], inplace=True)
table2.rename(columns={'OBS': 'Value'}, inplace=True)
table2['Value'] = table2['Value'].astype(int)
table2.drop(columns=['DATAMARKER'], inplace=True)
table2.head()

,Value,Geography,Nationality,Measure Type,Unit
0,9825,E12000001,TOTAL,Count,People
1,1985,E12000001,European Union EU15,Count,People
2,1200,E12000001,European Union EU8,Count,People
3,2152,E12000001,European Union EU2,Count,People
4,95,E12000001,European Union Other,Count,People


Table 3: NIN to adult Overseas Nationals Entering The UK by World region and nationality

In [12]:
savepreviewhtml(sheets[3])

Table 3 : National Insurance Number Registrations To Adult Overseas Nationals Entering The UK,,,,,,
Word region and nationality,,,,,,
,,Yr to March 2017,Yr to March 2018,,Difference,% Change
,,,,,,
Total,,785722.0,669846.0,,-115876.0,-0.1474770974975882
,,,,,,
,,,,,,
European Union,,593466.0,476785.0,,-116681.0,-0.19660940980612202
Non European Union,,191380.0,192273.0,,893.0,0.004666109311317797
,,,,,,
,,,,,,


In [13]:
observations = sheets[3].excel_ref('C29:D29').expand(DOWN).is_not_blank()
citizenship = sheets[3].excel_ref('B29').expand(DOWN).is_not_blank()
period = sheets[3].excel_ref('C3:D3')
period = period - period.regex('^INFO').expand(DOWN)
Dimensions = [
            HDim(period,'Period',DIRECTLY,ABOVE),
            HDim(citizenship,'Nationality', DIRECTLY, LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]
cs = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
savepreviewhtml(cs)

In [14]:
table3 = cs.topandas()
table3.head()

,OBS,DATAMARKER,Period,Nationality,Measure Type,Unit
0,181882,NaN,Yr to March 2017,Romania,Count,People
1,147956,NaN,Yr to March 2018,Romania,Count,People
2,83589,NaN,Yr to March 2017,Poland,Count,People
3,58370,NaN,Yr to March 2018,Poland,Count,People
4,61751,NaN,Yr to March 2017,Italy,Count,People


Drop non-numeric observations and ignore data markers.

Todo: figure out how to represent missing measured values.

In [15]:
import numpy as np
table3['OBS'].replace('', np.nan, inplace=True)
table3.dropna(subset=['OBS'], inplace=True)
table3.rename(columns={'OBS': 'Value'}, inplace=True)
table3['Value'] = table3['Value'].astype(int)
table3.drop(columns=['DATAMARKER'], inplace=True)
table3 = table3[['Period','Nationality','Measure Type','Value','Unit']]
table3.head()

,Period,Nationality,Measure Type,Value,Unit
0,Yr to March 2017,Romania,Count,181882,People
1,Yr to March 2018,Romania,Count,147956,People
2,Yr to March 2017,Poland,Count,83589,People
3,Yr to March 2018,Poland,Count,58370,People
4,Yr to March 2017,Italy,Count,61751,People


Convert periods to what look to be "Government years", see https://github.com/epimorphics/IntervalServer/blob/master/interval-uris.md#modern-government-business-intervals
    

In [16]:
year_to_re = re.compile('^Yr to March ([0-9]{4})$')
def to_govyear(s):
    match = year_to_re.match(s)
    assert match, f"Period '{s}' doesn't match pattern"
    y = match.group(1)
    return f'government-year/{int(y)-1}-{y}'
table3['Period'] = table3['Period'].map(to_govyear)
table3.head()

,Period,Nationality,Measure Type,Value,Unit
0,government-year/2016-2017,Romania,Count,181882,People
1,government-year/2017-2018,Romania,Count,147956,People
2,government-year/2016-2017,Poland,Count,83589,People
3,government-year/2017-2018,Poland,Count,58370,People
4,government-year/2016-2017,Italy,Count,61751,People


Consider each table as a slice with hidden constant dimension:

* `table1` needs `Geography` to be the UK, or `K02000001`
* `table2` needs `Period` to be year to March 2018, or `government-year/2017-2018`
* `table3` needs `Geography` to be the UK, or `K02000001`

In [17]:
table1['Geography'] = 'K02000001'
table2['Period'] = 'government-year/2017-2018'
table3['Geography'] = 'K02000001'
full_table = pd.concat([table1, table2, table3], sort=False)
full_table.head()

,Period,Nationality,Measure Type,Value,Unit,Geography
0,quarter/2002-Q1,TOTAL,Count,69701,People,K02000001
1,quarter/2002-Q1,European Union EU15,Count,18150,People,K02000001
2,quarter/2002-Q1,European Union EU8,Count,2217,People,K02000001
3,quarter/2002-Q1,European Union EU2,Count,1269,People,K02000001
4,quarter/2002-Q1,European Union Other,Count,455,People,K02000001


Derive the nationalities and groupings from the final tab of the spreadsheet

In [18]:
world_regions = sheets[4].filter('World region').fill(DOWN).is_not_blank()
sub_groups = world_regions.shift(RIGHT)
nationalities = sub_groups.shift(RIGHT)
savepreviewhtml([world_regions, sub_groups, nationalities])

item 0,item 1,item 2
Table 4 : World area reporting structure - subgroups and nationalities.,,
,,
"For reporting purposes, countries have been grouped into ‘world regions’ and ‘sub groups’. These are based on geographical location and status within the European Union. Countries are grouped according to current world area status for the entire back series. The following table shows the structural makeup of the sub groups and wider world areas:",,
,,
,,
World region,Sub group,Nationality
Asia,East Asia,China
Asia,East Asia,Hong Kong
Asia,East Asia,Japan
Asia,East Asia,Macau


In [19]:
regions = set(r.value for r in world_regions)
sub_group_parents = {}
for group in sub_groups:
    parent = group.shift(LEFT).value
    if group.value in sub_group_parents:
        assert sub_group_parents[group.value] == parent, 'sub group in multiple regions'
    else:
        sub_group_parents[group.value] = parent
nationality_parents = {}
for nationality in nationalities:
    parent = nationality.shift(LEFT).value
    if nationality.value in nationality_parents:
        assert nationality_parents[nationality.value] == parent, 'nationality in multiple sub groups'
    else:
        nationality_parents[nationality.value] = parent

codelist = [
    ('All nationalities', 'all', '')
] + [
    (region, pathify(region), 'all')
    for region in regions
] + [
    (sub_group, pathify(sub_group), pathify(parent))
    for sub_group, parent in sub_group_parents.items()
] + [
    (nationality, pathify(nationality), pathify(parent))
    for nationality, parent in nationality_parents.items()
] + [
    ('Unknown', 'unknown', 'all')   
]

codelist_df = pd.DataFrame.from_records(codelist, columns=('Label', 'Notation', 'Parent Notation'))
codelist_df['Sort Priority'] = codelist_df.index + 1
codelist_df['Description'] = ''
if not codelist_df['Notation'].is_unique:
    display(codelist_df[codelist_df.duplicated('Notation', keep='first')])
    assert False, "Notation not unique for nationalities codelist"
out = Path('out')
out.mkdir(exist_ok=True)
codelist_df.to_csv(out / 'nationalities.csv', index=False)

Use codes for nationality and check they exist in the above codelist

In [20]:
def notation(s):
    r = pathify(s)
    if r == 'total':
        return 'all'
    elif r == 'other-/-unknown':
        return 'unknown'
    elif r == 'st-helena-ascension-and-tristan-da-cunha':
        return 'st-helena'
    return r

full_table['Nationality'] = full_table['Nationality'].apply(notation)
used_nationalities = set(full_table['Nationality'].unique())
declared_nationalities = set(codelist_df['Notation'].unique())
assert used_nationalities.issubset(declared_nationalities), used_nationalities - declared_nationalities

One issue is that table 2 has for the last row 'Overseas registration'. For now, we'll split the output into two tables so that we can use two different definitions of the same component/dimension.

In [21]:
ons_geo_table = full_table[full_table['Geography'].notnull()]
overseas_table = full_table[~full_table['Geography'].notnull()].copy()
overseas_table['Registration Geography'] = 'Overseas'
overseas_table.drop(columns=['Geography'], inplace=True)

In [22]:
from pathlib import Path
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

ons_geo_table.to_csv(destinationFolder / ('ons_geo_observations.csv'), index = False)
overseas_table.to_csv(destinationFolder / ('overseas_observations.csv'), index = False)

In [23]:
from gssutils.metadata import THEME
scraper.dataset.family = 'migration'
scraper.dataset.theme = THEME['population']

with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())